> # Step 1: Pulling the data using Python IPEDS package







---


# **Up to 2020**
### Extracting data on Computing (CIP 11) Bachelor's degree completions by gender from 2015 acedemic year to 2020 from IPEDS (via Python library)

---



In [ ]:
pip install pypeds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.4/612.4 kB 24.9 MB/s eta 0:00:00


In [ ]:
from pypeds import ipeds
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
df_upto2020 = pd.DataFrame(columns=['academic_year', 'total_degrees', 'total_women', 'black_latina_aian_nhpi_women'])

In [ ]:
df_upto2020

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women


In [ ]:
def cs_degrees(YEAR):
  hd = ipeds.HD(years=[YEAR])
  hd.extract()
  df_hd = hd.load()
  c = ipeds.C_A(years=[YEAR])
  c.extract()
  df_c = c.load()
  df = df_hd.merge(df_c, on=['unitid', 'survey_year'])
  df['cipcode'] = df['cipcode'].astype(str)
  df['awlevel'] = df['awlevel'].astype(int)
  df = df[df['cipcode'].str.startswith('11')]       # cip 11
  df = df[df['awlevel'] == 5]                       # bachelors degrees only
  result = df.groupby('instnm').agg(
    total_degrees=('ctotalt', 'sum'),
    total_women=('ctotalw', 'sum'),
    cbkaaw=('cbkaaw', 'sum'),
    chispw=('chispw', 'sum'),
    caianw=('caianw', 'sum'),
    cnhpiw=('cnhpiw', 'sum')
  ).reset_index()
  result['black_latina_aian_nhpi_women'] = result['cbkaaw'] + result['chispw'] + result['caianw'] + result['cnhpiw']
  result = result.drop(columns=['cbkaaw', 'chispw', 'caianw', 'cnhpiw'])
  result.total_degrees = result.total_degrees.astype(int)
  result.total_women = result.total_women.astype(int)
  result.black_latina_aian_nhpi_women = result.black_latina_aian_nhpi_women.astype(int)
  df_upto2020.loc[len(df_upto2020)] = [YEAR, result.total_degrees.sum(), result.total_women.sum(), result.black_latina_aian_nhpi_women.sum()]

In [ ]:
for YEAR in list(range(2015,2021,1)):
  cs_degrees(YEAR)

In [ ]:
# test
df_upto2020

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women
0,2015,62029,11307,2657
1,2016,67381,12848,2815
2,2017,74703,14578,3179
3,2018,83497,17097,3494
4,2019,92525,19474,4015
5,2020,101565,22161,4349


> # Step 2: Downloading files from IPEDS Data Center



---


# **Beyond 2020**
### Manually uploading and transforming data from IPEDS [Data Center](https://nces.ed.gov/ipeds/datacenter/DataFiles.aspx?year=2023&surveyNumber=-1&sid=55dd2983-120a-4dec-a3d1-93b478a3b7aa&rtid=7) for the 2021, 2022, and 2023 academic years.
### Then rename files following the *YYYYInstitutions.csv* or *YYYYCompletions.csv* convention and upload them to the notebook (via left side panel).


---



### **First, validate this method:**
### Compare this method's output for 2020 data files (Completions + Institutional Characteristics)
### to the output for 2020 data in the previos method.


---



In [ ]:
df_hd_2020 = pd.read_csv('/content/2020Institutions_validate.csv', header=0, encoding_errors='ignore')

In [ ]:
df_hd_2020.columns = df_hd_2020.columns.str.lower()

In [ ]:
df_hd_2020.head(10)

,unitid,instnm,ialias,addr,city,stabbr,zip,fips,obereg,chfnm,chftitle,gentele,ein,duns,opeid,opeflag,webaddr,adminurl,faidurl,applurl,npricurl,veturl,athurl,disaurl,sector,iclevel,control,hloffer,ugoffer,groffer,hdegofr1,deggrant,hbcu,hospital,medical,tribal,locale,openpubl,act,newid,deathyr,closedat,cyactive,postsec,pseflag,pset4flg,rptmth,instcat,c18basic,c18ipug,c18ipgrd,c18ugprf,c18enprf,c18szset,c15basic,ccbasic,carnegie,landgrnt,instsize,f1systyp,f1sysnam,f1syscod,cbsa,cbsatype,csa,necta,countycd,countynm,cngdstcd,longitud,latitude,dfrcgid,dfrcuscg
0,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,636001109,197216455,100200,1,www.aamu.edu/,https://www.aamu.edu/admissions-aid/index.html,https://www.aamu.edu/admissions-aid/financial-...,https://www.aamu.edu/admissions-aid/undergradu...,www.aamu.edu/admissions-aid/tuition-fees/net-p...,,,https://www.aamu.edu/administrativeoffices/VAD...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,17,10,4,14,18,18,16,1,3,2,-2,-2,26620,1,290,-2,1089,Madison County,105,-86.568502,34.783368,109,1
1,100663,University of Alabama at Birmingham,,Administration Bldg Suite 1070,Birmingham,AL,35294-0110,1,5,Ray L. Watts,President,2059344011,636005396,063690705,105200,1,https://www.uab.edu/,https://www.uab.edu/students/admissions/,https://www.uab.edu/students/paying-for-college,https://ssb.it.uab.edu/pls/sctprod/zsapk003_ug...,https://uab.studentaidcalculator.com/survey.aspx,https://www.uab.edu/students/veterans,https://www.uab.edu/registrar/students,https://www.uab.edu/students/disability/,1,1,1,9,1,1,11,1,2,1,1,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,17,17,9,5,15,15,15,15,2,5,1,The University of Alabama System,101050,13820,1,142,-2,1073,Jefferson County,107,-86.799345,33.505697,95,1
2,100690,Amridge University,Southern Christian University Regions University,1200 Taylor Rd,Montgomery,AL,36117-3553,1,5,Michael C.Turner,President,33438738777550,237034324,126307792,2503400,1,www.amridgeuniversity.edu/,www.amridgeuniversity.edu/admissions/,www.amridgeuniversity.edu/financialaid/,https://www2.amridgeuniversity.edu/Amridge/Log...,www2.amridgeuniversity.edu:9091/,www.amridgeuniversity.edu/admissions/military/,,www.amridgeuniversity.edu/academics/,2,1,2,9,1,1,12,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,20,19,18,5,5,6,20,21,51,2,1,2,-2,-2,33860,1,388,-2,1101,Montgomery County,102,-86.174010,32.362609,126,2
3,100706,University of Alabama in Huntsville,UAH University of Alabama Huntsville,301 Sparkman Dr,Huntsville,AL,35899,1,5,Darren Dawson,President,2568246120,630520830,949687123,105500,1,www.uah.edu/,https://www.uah.edu/admissions,finaid.uah.edu/,register.uah.edu/,finaid.uah.edu/,www.uah.edu/admissions/graduate/financial-aid/...,www.uah.edu/heoa,www.uah.edu/health-and-wellness/disability-sup...,1,1,1,9,1,1,11,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,16,17,17,15,4,12,16,15,16,2,3,1,The University of Alabama System,101050,26620,1,290,-2,1089,Madison County,105,-86.640449,34.724557,99,2
4,100724,Alabama State University,,915 S Jackson Street,Montgomery,AL,36104-0271,1,5,Quinton T. Ross,President,3342294100,636001101,040672685,100500,1,www.alasu.edu/,www.alasu.edu/admissions/index.aspx,www.alasu.edu/undergraduate/expensesandfinanci...,www.alasu.edu/admissions/undergrad-admissions/...,www.alasu.edu/cost-aid/tuition-costs/net-price...,,www.alasu.edu/cost-aid/consumer-info,www.alasu.edu/academics/researchcenters/alabam...,1,1,1,9,1,1,11,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,19,13,13,10,3,14,19,18,21,2,2,2,-2,-2,33860,1,388,-2,1101,Montgomery County,107,-86.295677,32.364317,118,1
5,100733,University of Alabama System Office,,500 University Blvd. East,Tuscaloosa,AL,35401,1,5,Finis St. John IV,Chancellor,2053485122,636001138,808245794,800400,2,www.uasystem.edu/,,,,www.uasystem.edu/,,,www.uasystem.edu/,0,1,1,9,1,1,11,1,2,-2,-2,2,12,1,A,-2,-2,-2,1,1,1,1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,2,-2,1,The University of Alabama System,101050,46220,1,-2,

In [ ]:
df_c_2020 = pd.read_csv('/content/2020Completions_validate.csv', header=0, encoding_errors='ignore')
df_c_2020.columns = df_c_2020.columns.str.lower()

In [ ]:
df_c_2020.head(10)

,unitid,cipcode,majornum,awlevel,xctotalt,ctotalt,xctotalm,ctotalm,xctotalw,ctotalw,xcaiant,caiant,xcaianm,caianm,xcaianw,caianw,xcasiat,casiat,xcasiam,casiam,xcasiaw,casiaw,xcbkaat,cbkaat,xcbkaam,cbkaam,xcbkaaw,cbkaaw,xchispt,chispt,xchispm,chispm,xchispw,chispw,xcnhpit,cnhpit,xcnhpim,cnhpim,xcnhpiw,cnhpiw,xcwhitt,cwhitt,xcwhitm,cwhitm,xcwhitw,cwhitw,xc2mort,c2mort,xc2morm,c2morm,xc2morw,c2morw,xcunknt,cunknt,xcunknm,cunknm,xcunknw,cunknw,xcnralt,cnralt,xcnralm,cnralm,xcnralw,cnralw
0,100654,1.0999,1,5,R,3,Z,0,R,3,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,Z,0,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,1,Z,0,R,1,Z,0,Z,0,Z,0
1,100654,1.1001,1,5,R,7,R,2,R,5,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,7,R,2,R,5,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0
2,100654,1.1001,1,7,R,4,R,0,R,4,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,R,0,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,R,0,R,2
3,100654,1.1001,1,17,R,2,R,2,R,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,1,R,1,R,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,1,R,1,R,0
4,100654,1.9999,1,5,R,6,R,4,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,3,R,1,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,R,2,R,0,Z,0,Z,0,Z,0,R,1,R,1,R,0,Z,0,Z,0,Z,0
5,100654,1.9999,1,7,R,0,R,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,0,R,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0
6,100654,1.9999,1,17,R,3,R,1,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,3,R,1,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0
7,100654,3.0599,1,5,R,5,R,4,R,1,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,R,1,R,1,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,R,2,R,0,Z,0,Z,0,Z,0,R,1,R,1,R,0,Z,0,Z,0,Z,0
8,100654,4.0301,1,5,R,3,R,2,R,1,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,3,R,2,R,1,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0
9,100654,4.0301,1,7,R,8,R,4,R,4,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,7,R,4,R,3,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,1,R,0,R,1,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0


In [ ]:
df_2020 = df_hd_2020.merge(df_c_2020, on='unitid')

In [ ]:
df_2020

,unitid,instnm,ialias,addr,city,stabbr,zip,fips,obereg,chfnm,chftitle,gentele,ein,duns,opeid,opeflag,webaddr,adminurl,faidurl,applurl,npricurl,veturl,athurl,disaurl,sector,iclevel,control,hloffer,ugoffer,groffer,hdegofr1,deggrant,hbcu,hospital,medical,tribal,locale,openpubl,act,newid,deathyr,closedat,cyactive,postsec,pseflag,pset4flg,rptmth,instcat,c18basic,c18ipug,c18ipgrd,c18ugprf,c18enprf,c18szset,c15basic,ccbasic,carnegie,landgrnt,instsize,f1systyp,f1sysnam,f1syscod,cbsa,cbsatype,csa,necta,countycd,countynm,cngdstcd,longitud,latitude,dfrcgid,dfrcuscg,cipcode,majornum,awlevel,xctotalt,ctotalt,xctotalm,ctotalm,xctotalw,ctotalw,xcaiant,caiant,xcaianm,caianm,xcaianw,caianw,xcasiat,casiat,xcasiam,casiam,xcasiaw,casiaw,xcbkaat,cbkaat,xcbkaam,cbkaam,xcbkaaw,cbkaaw,xchispt,chispt,xchispm,chispm,xchispw,chispw,xcnhpit,cnhpit,xcnhpim,cnhpim,xcnhpiw,cnhpiw,xcwhitt,cwhitt,xcwhitm,cwhitm,xcwhitw,cwhitw,xc2mort,c2mort,xc2morm,c2morm,xc2morw,c2morw,xcunknt,cunknt,xcunknm,cunknm,xcunknw,cunknw,xcnralt,cnralt,xcnralm,cnralm,xcnralw,cnralw
0,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,636001109,197216455,100200,1,www.aamu.edu/,https://www.aamu.edu/admissions-aid/index.html,https://www.aamu.edu/admissions-aid/financial-...,https://www.aamu.edu/admissions-aid/undergradu...,www.aamu.edu/admissions-aid/tuition-fees/net-p...,,,https://www.aamu.edu/administrativeoffices/VAD...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,17,10,4,14,18,18,16,1,3,2,-2,-2,26620,1,290,-2,1089,Madison County,105,-86.568502,34.783368,109,1,1.0999,1,5,R,3,Z,0,R,3,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,Z,0,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,1,Z,0,R,1,Z,0,Z,0,Z,0
1,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,636001109,197216455,100200,1,www.aamu.edu/,https://www.aamu.edu/admissions-aid/index.html,https://www.aamu.edu/admissions-aid/financial-...,https://www.aamu.edu/admissions-aid/undergradu...,www.aamu.edu/admissions-aid/tuition-fees/net-p...,,,https://www.aamu.edu/administrativeoffices/VAD...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,17,10,4,14,18,18,16,1,3,2,-2,-2,26620,1,290,-2,1089,Madison County,105,-86.568502,34.783368,109,1,1.1001,1,5,R,7,R,2,R,5,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,7,R,2,R,5,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0
2,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,636001109,197216455,100200,1,www.aamu.edu/,https://www.aamu.edu/admissions-aid/index.html,https://www.aamu.edu/admissions-aid/financial-...,https://www.aamu.edu/admissions-aid/undergradu...,www.aamu.edu/admissions-aid/tuition-fees/net-p...,,,https://www.aamu.edu/administrativeoffices/VAD...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,17,10,4,14,18,18,16,1,3,2,-2,-2,26620,1,290,-2,1089,Madison County,105,-86.568502,34.783368,109,1,1.1001,1,7,R,4,R,0,R,4,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,R,0,R,2,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,2,R,0,R,2
3,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,636001109,197216455,100200,1,www.aamu.edu/,https://www.aamu.edu/admissions-aid/index.html,https://www.aamu.edu/admissions-aid/financial-...,https://www.aamu.edu/admissions-aid/undergradu...,www.aamu.edu/admissions-aid/tuition-fees/net-p...,,,https://www.aamu.edu/administrativeoffices/VAD...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,17,10,4,14,18,18,16,1,3,2,-2,-2,26620,1,290,-2,1089,Madison County,105,-86.568502,34.783368,109,1,1.1001,1,17,R,2,R,2,R,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,1,R,1,R,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,Z,0,R,1,R,1,R,0
4,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,6360011

In [ ]:
df_2020['cipcode'] = df_2020['cipcode'].astype(str)
df_2020['awlevel'] = df_2020['awlevel'].astype(int)
df_2020 = df_2020[df_2020['cipcode'].str.startswith('11')]       # cip 11
df_2020 = df_2020[df_2020['awlevel'] == 5]                       # bachelors degrees only
result = df_2020.groupby('instnm').agg(
    total_degrees=('ctotalt', 'sum'),
    total_women=('ctotalw', 'sum'),
    cbkaaw=('cbkaaw', 'sum'),
    chispw=('chispw', 'sum'),
    caianw=('caianw', 'sum'),
    cnhpiw=('cnhpiw', 'sum')
).reset_index()
result['black_latina_aian_nhpi_women'] = result['cbkaaw'] + result['chispw'] + result['caianw'] + result['cnhpiw']
result = result.drop(columns=['cbkaaw', 'chispw', 'caianw', 'cnhpiw'])
result.total_degrees = result.total_degrees.astype(int)
result.total_women = result.total_women.astype(int)
result.black_latina_aian_nhpi_women = result.black_latina_aian_nhpi_women.astype(int)
print(result.total_degrees.sum())
print(result.total_women.sum())
print(result.black_latina_aian_nhpi_women.sum())

101565
22161
4349




---


## **Check!**
### Numbers indeed match -> proceed with 2021, 2022, and 2023 years


---



In [ ]:
YEAR = 2021
completions_path = '/content/'+str(YEAR)+'Completions.csv'
print(completions_path)
institutions_path = '/content/'+str(YEAR)+'Institutions.csv'
print(institutions_path)

/content/2021Completions.csv
/content/2021Institutions.csv


In [ ]:
df_beyond2020 = pd.DataFrame(columns=['academic_year', 'total_degrees', 'total_women', 'black_latina_aian_nhpi_women'])
df_beyond2020

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women


In [ ]:
def cs_degrees_beyond2020(YEAR):
  institutions_path = '/content/'+str(YEAR)+'Institutions.csv'
  df_hd = pd.read_csv(institutions_path, header=0, encoding_errors='ignore')
  df_hd.columns = df_hd.columns.str.lower()
  completions_path = '/content/'+str(YEAR)+'Completions.csv'
  df_c = pd.read_csv(completions_path, header=0, encoding_errors='ignore')
  df_c.columns = df_c.columns.str.lower()
  df = df_hd.merge(df_c, on='unitid')
  df['cipcode'] = df['cipcode'].astype(str)
  df['awlevel'] = df['awlevel'].astype(int)
  df = df[df['cipcode'].str.startswith('11')]       # cip 11
  df = df[df['awlevel'] == 5]                       # bachelors degrees only
  result = df.groupby('instnm').agg(
    total_degrees=('ctotalt', 'sum'),
    total_women=('ctotalw', 'sum'),
    cbkaaw=('cbkaaw', 'sum'),
    chispw=('chispw', 'sum'),
    caianw=('caianw', 'sum'),
    cnhpiw=('cnhpiw', 'sum')
  ).reset_index()
  result['black_latina_aian_nhpi_women'] = result['cbkaaw'] + result['chispw'] + result['caianw'] + result['cnhpiw']
  result = result.drop(columns=['cbkaaw', 'chispw', 'caianw', 'cnhpiw'])
  result.total_degrees = result.total_degrees.astype(int)
  result.total_women = result.total_women.astype(int)
  result.black_latina_aian_nhpi_women = result.black_latina_aian_nhpi_women.astype(int)
  df_beyond2020.loc[len(df_beyond2020)] = [YEAR, result.total_degrees.sum(), result.total_women.sum(), result.black_latina_aian_nhpi_women.sum()]

In [ ]:
for YEAR in list(range(2021,2024,1)):
  cs_degrees_beyond2020(YEAR)

In [ ]:
df_beyond2020

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women
0,2021,109502,24476,5151
1,2022,113492,26181,5516
2,2023,118656,27737,6053


### **YAY!** Works for 2021-2023, and numbers seem reasonable (follow an established growth trend)

In [ ]:
df_computingdegrees = pd.concat([df_upto2020, df_beyond2020], ignore_index=True)

In [ ]:
df_computingdegrees

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women
0,2015,62029,11307,2657
1,2016,67381,12848,2815
2,2017,74703,14578,3179
3,2018,83497,17097,3494
4,2019,92525,19474,4015
5,2020,101565,22161,4349
6,2021,109502,24476,5151
7,2022,113492,26181,5516
8,2023,118656,27737,6053


> # Step 3: Repeating the above steps but only for top 25 universities



---


# **Perfect!**
## Let's repeat this process for the top 25 (actually, 26 as of the year 2024 lol) universities.
## Then build a table excluding them from the total computing degrees data.


---



In [ ]:
# test
df_2020[df_2020['unitid'] == 166629]

,unitid,instnm,ialias,addr,city,stabbr,zip,fips,obereg,chfnm,chftitle,gentele,ein,duns,opeid,opeflag,webaddr,adminurl,faidurl,applurl,npricurl,veturl,athurl,disaurl,sector,iclevel,control,hloffer,ugoffer,groffer,hdegofr1,deggrant,hbcu,hospital,medical,tribal,locale,openpubl,act,newid,deathyr,closedat,cyactive,postsec,pseflag,pset4flg,rptmth,instcat,c18basic,c18ipug,c18ipgrd,c18ugprf,c18enprf,c18szset,c15basic,ccbasic,carnegie,landgrnt,instsize,f1systyp,f1sysnam,f1syscod,cbsa,cbsatype,csa,necta,countycd,countynm,cngdstcd,longitud,latitude,dfrcgid,dfrcuscg,cipcode,majornum,awlevel,xctotalt,ctotalt,xctotalm,ctotalm,xctotalw,ctotalw,xcaiant,caiant,xcaianm,caianm,xcaianw,caianw,xcasiat,casiat,xcasiam,casiam,xcasiaw,casiaw,xcbkaat,cbkaat,xcbkaam,cbkaam,xcbkaaw,cbkaaw,xchispt,chispt,xchispm,chispm,xchispw,chispw,xcnhpit,cnhpit,xcnhpim,cnhpim,xcnhpiw,cnhpiw,xcwhitt,cwhitt,xcwhitm,cwhitm,xcwhitw,cwhitw,xc2mort,c2mort,xc2morm,c2morm,xc2morw,c2morw,xcunknt,cunknt,xcunknm,cunknm,xcunknw,cunknw,xcnralt,cnralt,xcnralm,cnralm,xcnralw,cnralw
104256,166629,University of Massachusetts-Amherst,UMass Amherst,374 Whitmore Building 181 Presidents Drive,Amherst,MA,01003,25,1,Kumble R. Subbaswamy,Chancellor,4135450111,43167352,145370792,222100,1,www.umass.edu/,www.umass.edu/admissions/,www.umass.edu/umfa/,www.commonapp.org/,www.umass.edu/umfa/sites/default/files/netpric...,www.umass.edu/bursar/veteran-educational-benefits,www.umass.edu/disclosure/,www.umass.edu/disability/,1,1,1,9,1,1,11,1,2,2,2,2,21,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,15,15,4,17,15,15,15,1,5,1,University of Massachusetts,125030,44140,1,-2,78100,25015,Hampshire County,2502,-72.526728,42.385999,94,1,11.0104,1,5,R,29,R,15,R,14,R,0,R,0,R,0,R,11,R,7,R,4,R,0,R,0,R,0,R,1,R,0,R,1,R,0,R,0,R,0,R,14,R,6,R,8,R,0,R,0,R,0,R,0,R,0,R,0,R,3,R,2,R,1
104257,166629,University of Massachusetts-Amherst,UMass Amherst,374 Whitmore Building 181 Presidents Drive,Amherst,MA,01003,25,1,Kumble R. Subbaswamy,Chancellor,4135450111,43167352,145370792,222100,1,www.umass.edu/,www.umass.edu/admissions/,www.umass.edu/umfa/,www.commonapp.org/,www.umass.edu/umfa/sites/default/files/netpric...,www.umass.edu/bursar/veteran-educational-benefits,www.umass.edu/disclosure/,www.umass.edu/disability/,1,1,1,9,1,1,11,1,2,2,2,2,21,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,15,15,4,17,15,15,15,1,5,1,University of Massachusetts,125030,44140,1,-2,78100,25015,Hampshire County,2502,-72.526728,42.385999,94,1,11.0104,2,5,R,1,R,0,R,1,R,0,R,0,R,0,R,1,R,0,R,1,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R,0
104258,166629,University of Massachusetts-Amherst,UMass Amherst,374 Whitmore Building 181 Presidents Drive,Amherst,MA,01003,25,1,Kumble R. Subbaswamy,Chancellor,4135450111,43167352,145370792,222100,1,www.umass.edu/,www.umass.edu/admissions/,www.umass.edu/umfa/,www.commonapp.org/,www.umass.edu/umfa/sites/default/files/netpric...,www.umass.edu/bursar/veteran-educational-benefits,www.umass.edu/disclosure/,www.umass.edu/disability/,1,1,1,9,1,1,11,1,2,2,2,2,21,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,15,15,4,17,15,15,15,1,5,1,University of Massachusetts,125030,44140,1,-2,78100,25015,Hampshire County,2502,-72.526728,42.385999,94,1,11.0701,1,5,R,308,R,275,R,33,R,0,R,0,R,0,R,69,R,61,R,8,R,2,R,2,R,0,R,9,R,8,R,1,R,1,R,1,R,0,R,147,R,136,R,11,R,7,R,5,R,2,R,18,R,15,R,3,R,55,R,47,R,8
104261,166629,University of Massachusetts-Amherst,UMass Amherst,374 Whitmore Building 181 Presidents Drive,Amherst,MA,01003,25,1,Kumble R. Subbaswamy,Chancellor,4135450111,43167352,145370792,222100,1,www.umass.edu/,www.umass.edu/admissions/,www.umass.edu/umfa/,www.commonapp.org/,www.umass.edu/umfa/sites/default/files/netpric...,www.umass.edu/bursar/veteran-educational-benefits,www.umass.edu/disclosure/,www.umass.edu/disability/,1,1,1,9,1,1,11,1,2,2,2,2,21,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,15,15,4,17,15,15,15,1,5,1,University of Massachusetts,125030,44140,1,-2,78100,25015,Hampshire County,2502,-72.526728,42.385999,94,1,11.0701,2,5,R,2,R,2,R,0,R,0,R,0,R,0,R,0,R,0,R,0,R

In [ ]:
df_top25universities = pd.read_csv('/content/top25_unitid.csv', header=0, encoding_errors='ignore')
df_top25universities.columns = df_top25universities.columns.str.lower()

In [ ]:
unitids_to_keep = list(df_top25universities['unitid'])
unitids_to_keep

[190415.0,
 236948.0,
 240444.0,
 130794.0,
 217156.0,
 110404.0,
 211440.0,
 198419.0,
 139755.0,
 166027.0,
 162928.0,
 166683.0,
 193900.0,
 186131.0,
 243780.0,
 243744.0,
 228778.0,
 110635.0,
 110662.0,
 110680.0,
 145637.0,
 163286.0,
 170976.0,
 199120.0,
 123961.0,
 nan,
 nan]

In [ ]:
df_upto2020_top25 = pd.DataFrame(columns=['academic_year', 'total_degrees', 'total_women', 'black_latina_aian_nhpi_women'])

In [ ]:
def cs_degrees_top25(YEAR):
  hd = ipeds.HD(years=[YEAR])
  hd.extract()
  df_hd = hd.load()
  c = ipeds.C_A(years=[YEAR])
  c.extract()
  df_c = c.load()
  df = df_hd.merge(df_c, on=['unitid', 'survey_year'])
  df['cipcode'] = df['cipcode'].astype(str)
  df['awlevel'] = df['awlevel'].astype(int)
  df = df[df['cipcode'].str.startswith('11')]       # cip 11
  df = df[df['awlevel'] == 5]                       # bachelors degrees only
  df = df[df['unitid'].isin(unitids_to_keep)]
  result = df.groupby('instnm').agg(
    total_degrees=('ctotalt', 'sum'),
    total_women=('ctotalw', 'sum'),
    cbkaaw=('cbkaaw', 'sum'),
    chispw=('chispw', 'sum'),
    caianw=('caianw', 'sum'),
    cnhpiw=('cnhpiw', 'sum')
  ).reset_index()
  result['black_latina_aian_nhpi_women'] = result['cbkaaw'] + result['chispw'] + result['caianw'] + result['cnhpiw']
  result = result.drop(columns=['cbkaaw', 'chispw', 'caianw', 'cnhpiw'])
  result.total_degrees = result.total_degrees.astype(int)
  result.total_women = result.total_women.astype(int)
  result.black_latina_aian_nhpi_women = result.black_latina_aian_nhpi_women.astype(int)
  df_upto2020_top25.loc[len(df_upto2020_top25)] = [YEAR, result.total_degrees.sum(), result.total_women.sum(), result.black_latina_aian_nhpi_women.sum()]

In [ ]:
for YEAR in list(range(2015,2021,1)):
  cs_degrees_top25(YEAR)

In [ ]:
df_upto2020_top25

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women
0,2015,5376,1163,82
1,2016,6660,1546,110
2,2017,7822,1959,145
3,2018,9211,2410,184
4,2019,10647,3018,249
5,2020,12514,3756,325


In [ ]:
df_beyond2020_top25 = pd.DataFrame(columns=['academic_year', 'total_degrees', 'total_women', 'black_latina_aian_nhpi_women'])
df_beyond2020_top25

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women


In [ ]:
def cs_degrees_beyond2020_top25(YEAR):
  institutions_path = '/content/'+str(YEAR)+'Institutions.csv'
  df_hd = pd.read_csv(institutions_path, header=0, encoding_errors='ignore')
  df_hd.columns = df_hd.columns.str.lower()
  completions_path = '/content/'+str(YEAR)+'Completions.csv'
  df_c = pd.read_csv(completions_path, header=0, encoding_errors='ignore')
  df_c.columns = df_c.columns.str.lower()
  df = df_hd.merge(df_c, on='unitid')
  df['cipcode'] = df['cipcode'].astype(str)
  df['awlevel'] = df['awlevel'].astype(int)
  df = df[df['cipcode'].str.startswith('11')]       # cip 11
  df = df[df['awlevel'] == 5]                       # bachelors degrees only
  df = df[df['unitid'].isin(unitids_to_keep)]
  result = df.groupby('instnm').agg(
    total_degrees=('ctotalt', 'sum'),
    total_women=('ctotalw', 'sum'),
    cbkaaw=('cbkaaw', 'sum'),
    chispw=('chispw', 'sum'),
    caianw=('caianw', 'sum'),
    cnhpiw=('cnhpiw', 'sum')
  ).reset_index()
  result['black_latina_aian_nhpi_women'] = result['cbkaaw'] + result['chispw'] + result['caianw'] + result['cnhpiw']
  result = result.drop(columns=['cbkaaw', 'chispw', 'caianw', 'cnhpiw'])
  result.total_degrees = result.total_degrees.astype(int)
  result.total_women = result.total_women.astype(int)
  result.black_latina_aian_nhpi_women = result.black_latina_aian_nhpi_women.astype(int)
  df_beyond2020_top25.loc[len(df_beyond2020_top25)] = [YEAR, result.total_degrees.sum(), result.total_women.sum(), result.black_latina_aian_nhpi_women.sum()]

In [ ]:
for YEAR in list(range(2021,2024,1)):
  cs_degrees_beyond2020_top25(YEAR)

In [ ]:
df_beyond2020_top25

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women
0,2021,13580,4212,380
1,2022,14569,4552,403
2,2023,15649,4988,481


In [ ]:
df_computingdegrees_top25 = pd.concat([df_upto2020_top25, df_beyond2020_top25], ignore_index=True)

In [ ]:
df_computingdegrees_top25

,academic_year,total_degrees,total_women,black_latina_aian_nhpi_women
0,2015,5376,1163,82
1,2016,6660,1546,110
2,2017,7822,1959,145
3,2018,9211,2410,184
4,2019,10647,3018,249
5,2020,12514,3756,325
6,2021,13580,4212,380
7,2022,14569,4552,403
8,2023,15649,4988,481




---








# **Done!**
### Now just export the tables in a spreadsheet and calculate related percentages and forecast data!
### Link to [my spreadsheet](https://docs.google.com/spreadsheets/d/1c-1CxTrlulq8hLJI-P_QH2SObuinHF5yYy15MehsaQU/edit?usp=sharing)








---



In [ ]:
# working code template:



'''
def cs_degrees(YEAR):
  hd = ipeds.HD(years=[YEAR])
  hd.extract()
  df_hd = hd.load()
  c = ipeds.C_A(years=[YEAR])
  c.extract()
  df_c = c.load()
  df = df_hd.merge(df_c, on=['unitid', 'survey_year'])
  df['cipcode'] = df['cipcode'].astype(str)
  df['awlevel'] = df['awlevel'].astype(int)
  df = df[df['cipcode'].str.startswith('11')]       # cip 11
  df = df[df['awlevel'] == 5]                       # bachelors degrees only
  result = df.groupby('instnm').agg(
    total_degrees=('ctotalt', 'sum'),
    total_women=('ctotalw', 'sum'),
    cbkaaw=('cbkaaw', 'sum'),
    chispw=('chispw', 'sum'),
    caianw=('caianw', 'sum'),
    cnhpiw=('cnhpiw', 'sum')
  ).reset_index()
  result['black_latina_aian_nhpi_women'] = result['cbkaaw'] + result['chispw'] + result['caianw'] + result['cnhpiw']
  result = result.drop(columns=['cbkaaw', 'chispw', 'caianw', 'cnhpiw'])
  result.total_degrees = result.total_degrees.astype(int)
  result.total_women = result.total_women.astype(int)
  result.black_latina_aian_nhpi_women = result.black_latina_aian_nhpi_women.astype(int)
  print(result.total_degrees.sum())
  print(result.total_women.sum())
  print(result.black_latina_aian_nhpi_women.sum())
  '''